In [1]:
import numpy as np 

# Set seed values
seed = 42
#random.seed = seed
np.random.seed(seed=seed)

## LOADING DATA FROM S3

In [2]:
import boto3

_BUCKET_NAME = 'shipping-ml'
_PREFIX = 'data/raw/daily/raw/DataGuido/train_image/'
_BUFFER = 1000

s3 = boto3.resource('s3')
my_bucket = s3.Bucket(_BUCKET_NAME)

def ListFiles(my_bucket, buffer=_BUFFER):
    lista_archivos = []
    i = 0
    for object_summary in my_bucket.objects.filter(Prefix="data/raw/daily/raw/DataGuido/train_image/"):
        lista_archivos.append(object_summary.key)
        #print(object_summary.key)
        i = i + 1
        if i % buffer == 0:
            temp = lista_archivos
            lista_archivos = []
            print('fin ciclo')
            i = 0
            yield temp
                

In [3]:
import os
import glob
import numpy as np 
from PIL import Image

train_images = '/home/ec2-user/SageMaker/project/ACDC/train_image/'
train_mask   = '/home/ec2-user/SageMaker/project/ACDC/train_mask/'
val_images   = '/home/ec2-user/SageMaker/project/ACDC/val_image/'
val_mask     = '/home/ec2-user/SageMaker/project/ACDC/val_mask/'
s3_train_msk = 'data/raw/daily/raw/DataGuido/train_label/'

#recorro S3
file_list = ListFiles(my_bucket, 100)
X_data = []
Y_data = []

for lista in file_list:
    
    files = glob.glob(train_images + '*.tif')
    for f in files:
        os.remove(f)
    files = glob.glob(train_mask + '*.tif')
    for f in files:
        os.remove(f)

        
    #guardo 1000 imagenes temporalmente
    for i, file in enumerate(lista):
        my_bucket.download_file(Key=file, Filename=train_images+'image{}.tif'.format(str(i)))
        msk_name = file.split('/')[-1].replace('img_patch', 'msk_patch')
        key_msk = s3_train_msk + msk_name
        my_bucket.download_file(Key=key_msk, Filename=train_mask+'image{}.tif'.format(str(i)))
    print('Loaded {} images from S3'.format(str(i)))
        
    #levanto las 1000 imagenes en memoria
    img_paths = [train_images+ id for id in os.listdir(train_images)]
    img_paths.sort()
    X_batch = np.array([np.asarray(Image.open(path)) for path in img_paths])
    
    img_paths = [train_mask + id for id in os.listdir(train_mask)]
    img_paths.sort()
    Y_batch = np.array([np.asarray(Image.open(path)) for path in img_paths])

    if len(X_data) == 0:
        X_data = X_batch
        Y_data = Y_batch
    else:
        X_data = np.append(X_data, X_batch, axis=0)
        Y_data = np.append(Y_data, Y_batch, axis=0)
        break
        
Y_data = Y_data[:,:,:,np.newaxis]

fin ciclo
Loaded 99 images from S3
fin ciclo
Loaded 99 images from S3


In [4]:
import os

img_paths = [val_images + id for id in os.listdir(val_images)]
img_paths.sort()
X_val = np.array([np.asarray(Image.open(path)) for path in img_paths])

img_paths = [val_mask + id for id in os.listdir(val_mask)]
img_paths.sort()
Y_val = np.array([np.asarray(Image.open(path)) for path in img_paths])
Y_val = Y_val[:,:,:,np.newaxis]

## ONLINE GENERATORS

In [5]:
from keras.preprocessing.image import ImageDataGenerator

# Image data generator distortion options
data_gen_args = dict(rotation_range=45.,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     shear_range=0.2,
                     zoom_range=0.2,
                     horizontal_flip=True,
                     vertical_flip=True,
                     fill_mode='reflect')  #use 'constant'??

def adjustData(img, mask):
    img = img / 255.
    mask = mask / 255.
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0
    return (img, mask)

# Runtime data augmentation
def flow_images(X, Y, batch_size=100, seed=None, data_gen_args=None):
        
    # Train data, provide the same seed and keyword arguments to the fit and flow methods
    if data_gen_args:
        X_datagen = ImageDataGenerator(**data_gen_args)
        Y_datagen = ImageDataGenerator(**data_gen_args)
    else:
        X_datagen = ImageDataGenerator()
        Y_datagen = ImageDataGenerator()
    
    X_datagen.fit(X, augment=True, seed=seed)
    Y_datagen.fit(Y, augment=True, seed=seed)
    X_generator = X_datagen.flow(X, batch_size=batch_size, shuffle=True, seed=seed)
    Y_generator = Y_datagen.flow(Y, batch_size=batch_size, shuffle=True, seed=seed)
    
    # combine generators into one which yields image and masks
    generator = zip(X_generator, Y_generator)
    
    #for (img, mask) in generator:
    #    print(img.shape)
    #    img, mask = adjustData(img,mask)
    return generator




Using TensorFlow backend.


## LOSS FUNCTION

In [6]:
smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


## U-NET MODEL

In [7]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
import keras

# Design our model architecture here
def unet_model(img_width=256, img_height=256):
    '''
    Modified from https://keunwoochoi.wordpress.com/2017/10/11/u-net-on-keras-2-0/
    '''
    n_ch_exps = [4, 5, 6, 7, 8, 9]   #the n-th deep channel's exponent i.e. 2**n 16,32,64,128,256
    n_ch_exps = [4, 5, 6]   #the n-th deep channel's exponent i.e. 2**n 16,32,64,128,256

    k_size = (3, 3)                  #size of filter kernel
    k_init = 'he_normal'             #kernel initializer

    if K.image_data_format() == 'channels_first':
        ch_axis = 1
        input_shape = (3, img_width, img_height)
    elif K.image_data_format() == 'channels_last':
        ch_axis = 3
        input_shape = (img_width, img_height, 3)

    inp = Input(shape=input_shape)
    encodeds = []

    # encoder
    enc = inp
    print(n_ch_exps)
    for l_idx, n_ch in enumerate(n_ch_exps):
        enc = Conv2D(filters=2**n_ch, kernel_size=k_size, activation='relu', padding='same', kernel_initializer=k_init)(enc)
        enc = Dropout(0.1*l_idx,)(enc)
        enc = Conv2D(filters=2**n_ch, kernel_size=k_size, activation='relu', padding='same', kernel_initializer=k_init)(enc)
        encodeds.append(enc)
        #print(l_idx, enc)
        if n_ch < n_ch_exps[-1]:  #do not run max pooling on the last encoding/downsampling step
            enc = MaxPooling2D(pool_size=(2,2))(enc)
    
    # decoder
    dec = enc
    print(n_ch_exps[::-1][1:])
    decoder_n_chs = n_ch_exps[::-1][1:]
    for l_idx, n_ch in enumerate(decoder_n_chs):
        l_idx_rev = len(n_ch_exps) - l_idx - 2  #
        dec = Conv2DTranspose(filters=2**n_ch, kernel_size=k_size, strides=(2,2), activation='relu', padding='same', kernel_initializer=k_init)(dec)
        dec = concatenate([dec, encodeds[l_idx_rev]], axis=ch_axis)
        dec = Conv2D(filters=2**n_ch, kernel_size=k_size, activation='relu', padding='same', kernel_initializer=k_init)(dec)
        dec = Dropout(0.1*l_idx)(dec)
        dec = Conv2D(filters=2**n_ch, kernel_size=k_size, activation='relu', padding='same', kernel_initializer=k_init)(dec)

    outp = Conv2DTranspose(filters=1, kernel_size=k_size, activation='sigmoid', padding='same', kernel_initializer='glorot_normal')(dec)

    model = Model(inputs=[inp], outputs=[outp])
    
    return model

In [8]:
# Set some model compile parameters
optimizer = 'adam'
loss      = dice_coef_loss
metrics   = [dice_coef]
num_gpus = 0

# Compile our model
model = unet_model(img_width=256, img_height=256)
model.summary()

# For more GPUs
if num_gpus > 1:
    model = multi_gpu_model(model, gpus=num_gpus)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

[4, 5, 6]
[5, 4]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 256, 256, 16) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 16) 2320        dropout_1[0][0]                  
____________________________________________________________________________________________

In [12]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest',
                    )

train_gen = flow_images(X_data, Y_data, batch_size=10, data_gen_args=data_gen_args, seed=seed)

val_gen = flow_images(X_val, Y_val, batch_size=10, data_gen_args=None, seed=seed)

In [14]:
model_checkpoint = ModelCheckpoint('unet_CMR.hdf5', monitor='loss', verbose=1, save_best_only=True)


history = model.fit_generator(train_gen, 
                              verbose=1,
                              steps_per_epoch=20, 
                              epochs=10,
                              callbacks=[model_checkpoint],
                              validation_data=val_gen, 
                              validation_steps=4
                             )


Epoch 1/10
20/20 [==============================] - 38s 2s/step - loss: -0.6018 - dice_coef: 0.6018 - val_loss: -0.8072 - val_dice_coef: 0.8072

Epoch 00001: loss improved from inf to -0.60180, saving model to unet_CMR.hdf5
Epoch 2/10
20/20 [==============================] - 35s 2s/step - loss: -0.6140 - dice_coef: 0.6140 - val_loss: -0.7429 - val_dice_coef: 0.7429

Epoch 00002: loss improved from -0.60180 to -0.61401, saving model to unet_CMR.hdf5
Epoch 3/10
20/20 [==============================] - 35s 2s/step - loss: -0.6122 - dice_coef: 0.6122 - val_loss: -0.7550 - val_dice_coef: 0.7550

Epoch 00003: loss did not improve from -0.61401
Epoch 4/10
 9/20 [============>.................] - ETA: 18s - loss: -0.6191 - dice_coef: 0.6191

KeyboardInterrupt: 